# REST Inference

## Setup

Verify that following variable settings match your deployed model's resource name and rest URL. The following code assumes that the kube service is in the same namespace, but you could refer to it in full with the namespace, for example: `http://modelmesh-serving.project-name.svc.cluster.local:8008/v2/models/fraud/infer`

In [33]:
deployed_model_name = "mohsin_calorie"
rest_url = "http://modelmesh-serving.mohsin-dvl-dev:8008"
infer_url = f"{rest_url}/v2/models/{deployed_model_name}/infer"

## Request Function

Build and submit the REST request. 

Note: You submit the data in the same format that you used for an ONNX inference.

In [29]:
import requests

def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "keras_tensor",
                "shape": [1, 20],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data)
    response_dict = response.json()
    print(response_dict)
    return response_dict

In [34]:
#Load the scaler
import pickle
with open('artifact/column_transformer.pkl', 'rb') as handle:
    ct = pickle.load(handle)

In [46]:
import pandas as pd

# Define your input data as a list
data = ['Tuesday', 30, 'Female', 43, 175, 'No', 'Calesthenics', 'Cardio', 272,
        37, 'Low', 'Yes', 'No', 'Weight Gain', 17, 5, 1821, 227, 91, 60]

# Create a DataFrame with the same structure as the data used during training
column_names = ['day', 'age', 'gender', 'weight', 'height_in_cms', 'digestive_disorder',
       'type_of_workout', 'additional_activity', 'total_calories_burned',
       'body_temperature', 'workout_intensity', 'followed_diet_plan',
       'taking_supplements', 'fitness_goal', 'target_weight',
       'target_time_in_months', 'actual_calorie_intake_in_kcal_per_day',
       'actual_carb_intake_in_gms', 'actual_protein_intake_in_gms',
       'actual_fat_intake_in_gms']

# Convert the list into a DataFrame
df = pd.DataFrame([data], columns=column_names)

In [47]:
# Transform the DataFrame using the column transformer
transformed_data = ct.transform(df)


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [50]:
# Convert the transformed data (NumPy array) to a Python list
transformed_data_list = transformed_data.tolist()
transformed_data_list

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [51]:
import json


# If the API expects the data in a specific structure, create a dictionary
json_data = {
    "inputs": [
        {
            "name": "keras_tensor",  # Assuming this is the expected input name
            "shape": [1, len(transformed_data_list[0])],  # Update shape based on your data
            "datatype": "FP32",  # Assuming the data type expected by your API
            "data": transformed_data_list[0]  # Send the first transformed data row
        }
    ]
}

# Convert the dictionary to a JSON string (optional step, since requests handles this)
json_payload = json.dumps(json_data)

# Now json_data is ready to be sent to your API
print(json_payload)


{"inputs": [{"name": "keras_tensor", "shape": [1, 11], "datatype": "FP32", "data": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}]}


In [53]:
# Pass the transformed data to the REST API
prediction = rest_request(json_payload)
print(prediction)

{'code': 3, 'message': 'invalid tensor data: not a json array'}
{'code': 3, 'message': 'invalid tensor data: not a json array'}


In [26]:
data = ['Tuesday', 30, 'Female', 43, 175, 'No', 'Calesthenics', 'Cardio', 272,
       37, 'Low', 'Yes', 'No', 'Weight Gain', 17, 5, 1821, 227, 91, 60]
prediction = rest_request(ct.transform([data]).tolist()[0])
prediction

ValueError: Specifying the columns using strings is only supported for dataframes.

In [16]:
import pandas as pd
df = pd.read_csv('X_test.csv')
data = ct.transform(df).tolist()[0]

In [21]:
prediction = rest_request(data)
prediction

{'code': 5, 'message': 'vmodel not found'}


{'code': 5, 'message': 'vmodel not found'}

In [21]:
threshhold = 0.95

if (prediction[0] > threshhold):
    print('fraud')
else:
    print('not fraud')

not fraud


## Example 1: user buys a coffee

In this example, the user is buying a coffee. The parameters given to the model are:
* same location as the last transaction (distance=0)
* same median price as the last transaction (ratio_to_median=1)
* using a pin number (pin=1)
* using the credit card chip (chip=1)
* not an online transaction (online=0)

In [13]:
data = [0.0, 1.0, 1.0, 1.0, 0.0]
prediction = rest_request(scaler.transform([data]).tolist()[0])
threshhold = 0.95

if (prediction[0] > threshhold):
    print('The model predicts that this is fraud')
else:
    print('The model predicts that this is not fraud')

{'model_name': 'fraud__isvc-64afb3686a', 'model_version': '1', 'outputs': [{'name': 'dense_1', 'datatype': 'FP32', 'shape': [1, 1], 'data': [1.7880393e-15]}]}
The model predicts that this is not fraud


## Example 2: fraudulent transaction

In this example, someone stole the user's credit card and is buying something online. The parameters given to the model are:
* very far away from the last transaction (distance=100)
* median price similar to the last transaction (ratio_to_median=1.2)
* not using a pin number (pin=0)
* not using the credit card chip (chip=0)
* is an online transaction (online=1)

In [14]:
data = [100, 1.2, 0.0, 0.0, 1.0]
prediction = rest_request(scaler.transform([data]).tolist()[0])
threshhold = 0.95

if (prediction[0] > threshhold):
    print('The model predicts that this is fraud')
else:
    print('The model predicts that this is not fraud')

{'model_name': 'fraud__isvc-64afb3686a', 'model_version': '1', 'outputs': [{'name': 'dense_1', 'datatype': 'FP32', 'shape': [1, 1], 'data': [0.9911938]}]}
The model predicts that this is fraud
